# SQL Subqueries - Lab Assignment #2

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="img/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data2.sqlite` in the data folder.

In [9]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [11]:
# Your code here; create the connection
conn = sqlite3.Connection("data/data2.sqlite")

### Q.1 Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [16]:
# Your code here
q0 = """
select customerNumber,
    contactLastName,
    contactFirstName
from customers
where customerNumber in (
    select customerNumber
    from orders
    where orderDate = '2003-01-31'
        )
;
"""

pd.read_sql(q0, conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


### Q.2 Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [33]:
# Your code here
q1 = """
SELECT p.productName as productname,
        od.quantityOrdered as totalnumberoforders
        
From products as p
inner join orderdetails as od
    on p.productCode = od.productCode
where od.quantityOrdered in (select quantityOrdered
                            from orderdetails
                            group by quantityOrdered
                            )
                            
group by productName
;
"""

q1_result = pd.read_sql(q1, conn)
q1_result

,productname,totalnumberoforders
0,18th Century Vintage Horse Carriage,25
1,18th century schooner,43
2,1900s Vintage Bi-Plane,49
3,1900s Vintage Tri-Plane,26
4,1903 Ford Model A,22
...,...,...
104,The Mayflower,29
105,The Queen Mary,22
106,The Schooner Bluenose,41
107,The Titanic,39


### Q.3 Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

**A quick note on the SQL  `SELECT DISTINCT` statement:**

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [39]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
q1 = """
select distinct p.productName as ProductName, sum(o.customerNumber) as TotalOrdered
from products as p
inner join orderdetails as od
    on p.productCode = od.productCode
inner join orders as o
    on od.orderNumber = o.orderNumber
where o.customerNumber in (select customerNumber
                            from orders
                            )
group by p.productName
order by sum(o.customerNumber) desc
;
"""

q1_result = pd.read_sql(q1, conn)
q1_result

,ProductName,TotalOrdered
0,1992 Ferrari 360 Spider red,13248
1,1900s Vintage Bi-Plane,9383
2,1980s Black Hawk Helicopter,9255
3,1937 Horch 930V Limousine,9169
4,1941 Chevrolet Special Deluxe Cabriolet,8844
...,...,...
104,1957 Ford Thunderbird,5904
105,2002 Chevy Corvette,5679
106,1969 Chevrolet Camaro Z28,5488
107,1949 Jaguar XK 120,5488


### Q.4 Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [45]:
# Your code here
q1 = """
select distinct e.employeeNumber
    ,e.firstName
    ,e.lastName
    ,off.city
    ,e.officeCode
    
from employees as e
inner join offices as off
    on e.officeCode = off.officeCode 
;
"""

q1_result = pd.read_sql(q1, conn)
q1_result

,employeeNumber,firstName,lastName,city,officeCode
0,1002,Diane,Murphy,San Francisco,1
1,1056,Mary,Patterson,San Francisco,1
2,1076,Jeff,Firrelli,San Francisco,1
3,1088,William,Patterson,Sydney,6
4,1102,Gerard,Bondur,Paris,4
5,1143,Anthony,Bow,San Francisco,1
6,1165,Leslie,Jennings,San Francisco,1
7,1166,Leslie,Thompson,San Francisco,1
8,1188,Julie,Firrelli,Boston,2
9,1216,Steve,Patterson,Boston,2


### Q.5 Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [53]:
# Your code here
q1 = """
select e.employeeNumber
        ,e.firstName
        ,e.lastName,
        count(c.customerNumber) as number_of_customers
from employees as e
join customers as c 
    on e.employeeNumber = c.salesRepEmployeeNumber
group by e.employeeNumber, e.firstName, e.lastName
having avg(c.creditLimit) > 15000

;
"""

q1_result = pd.read_sql(q1, conn)
q1_result

,employeeNumber,firstName,lastName,number_of_customers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!